In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
import math 
import re
import operator
import time
import json
import utils

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from keras.models import load_model
from keras_preprocessing.sequence import pad_sequences

In [2]:
# load the trained model
MODEL_PATH = 'moonboard_names_model_128'
NAME_MODEL = models.load_model(MODEL_PATH+'/model.h5')

In [3]:

params = json.load(open(MODEL_PATH + '/params.json'))
SEQUENCE_LENGTH = 128
char_to_int = params['char_to_int']
int_to_char = params['int_to_char']

#labels = {v: k for k, v in params['labels_index'].items()}

In [4]:
END_TOKEN = char_to_int['|']

In [5]:
problem_start = 'A5,B8,D12,F14,I18'
texts = [problem_start]

In [6]:
# let's pick some random problems to predict
cwd = os.getcwd()
print(cwd)
parent_wd = cwd.replace('/names', '')
raw_data_path = os.path.join(parent_wd, 'raw_data/moonGen_scrape_2016')

/Users/markliu/moonboard/names


In [7]:
with open(raw_data_path, 'rb') as f:
    raw_data = pickle.load(f)

In [8]:
print(list(raw_data.items())[3][1])

{'problem_name': 'GULAG', 'info': ['micklevin', '1 climber has repeated this problem', '7A', 'Feet follow hands', ''], 'url': 'https://moonboard.com/Problems/View/367885/gulag', 'num_empty': 1, 'num_stars': 2, 'moves': [{'Id': 2119166, 'Description': 'A5', 'IsStart': True, 'IsEnd': False}, {'Id': 2119167, 'Description': 'B8', 'IsStart': False, 'IsEnd': False}, {'Id': 2119168, 'Description': 'D12', 'IsStart': False, 'IsEnd': False}, {'Id': 2119169, 'Description': 'F14', 'IsStart': False, 'IsEnd': False}, {'Id': 2119170, 'Description': 'I18', 'IsStart': False, 'IsEnd': True}], 'grade': '7A', 'UserGrade': None, 'isBenchmark': False, 'repeats': 1, 'ProblemType': None, 'IsMaster': False, 'setter': {'Id': '05f91964-f58c-4a49-af50-abec02f71300', 'Nickname': 'micklevin', 'Firstname': 'Mick', 'Lastname': 'Levin', 'City': 'Halifax', 'Country': 'Canada', 'ProfileImageUrl': '/Content/Account/Users/Profile/05f91964-f58c-4a49-af50-abec02f71300.jpg?637231996827206508', 'CanShareData': True}}


In [9]:
def convert_problem_to_input(holds):
    # encode each problem as csv list of holds, followed by space, then name of the problem
    inp = []
    hold_string = ','.join([hold['Description'] for hold in holds])

    return hold_string

In [10]:
problem = list(raw_data.items())[3][1]

convert_problem_to_input(problem['moves'])

'A5,B8,D12,F14,I18'

In [101]:
texts = [convert_problem_to_input(x[1]['moves']) for x in list(raw_data.items())[0:5]]
print(len(texts))

5


In [103]:
def sample_from_prob_vec(pr):
    prob_thresh = np.max(pr,1) * 0.1
    pr[pr < prob_thresh]=0
    pr = (pr/pr.sum(axis=1,keepdims=1))[0]
    idxs = np.arange(len(pr))
    return np.random.choice(idxs, 1, p=pr)[0]

def name_texts(texts):
    MAX_LENGTH = 128
    # for each text, continue predicting until we reach max length or end token
    final_texts = np.array(texts)
    #while True:
    contin = [True] * len(texts)
    while any(contin):
        print(len(contin))
        sequences = utils.texts_to_sequences(final_texts[contin], char_to_int)
        print(len(sequences))
        data = pad_sequences(sequences, maxlen=SEQUENCE_LENGTH)
        predictions_list = NAME_MODEL.predict(data)
        #print(predictions_list)
        #chs = [int_to_char[x] for x in np.argmax(predictions_list,1)]
        chs = [int_to_char[sample_from_prob_vec(pr)] for pr in predictions_list]
        print(chs)
        final_texts = np.core.defchararray.add(final_texts[contin],chs)
        contin = [text[-1] != '|' and len(text) < MAX_LENGTH for text in final_texts]
        print(contin)
    return final_texts


In [104]:
final_texts = name_texts(texts)

5
5
1/1 [==============================] - 0s 16ms/step


AxisError: axis 1 is out of bounds for array of dimension 1

In [105]:
def name_text(text):
    texts = [text]
    #single 
    MAX_LENGTH = 128
    # for each text, continue predicting until we reach max length or end token
    final_texts = texts
    while True:
    
        sequences = utils.texts_to_sequences(final_texts, char_to_int)
        data = pad_sequences(sequences, maxlen=SEQUENCE_LENGTH)
        predictions_list = NAME_MODEL.predict(data)
        # print(predictions_list)
        # sample
        
        
        
        ch = sample_from_prob_vec(predictions_list)
        if len(sequences[0]) > MAX_LENGTH or ch == END_TOKEN:
            break
        print(int_to_char[ch])
        final_texts[0]+=int_to_char[ch]
        print(final_texts)
    print(text)

In [107]:
name_text("F5,H8,H10,H13,F15,A18,I4 ")

1/1 [==============================] - 0s 16ms/step
W
['F5,H8,H10,H13,F15,A18,I4 W']
1/1 [==============================] - 0s 16ms/step
A
['F5,H8,H10,H13,F15,A18,I4 WA']
1/1 [==============================] - 0s 16ms/step
R
['F5,H8,H10,H13,F15,A18,I4 WAR']
1/1 [==============================] - 0s 15ms/step
M
['F5,H8,H10,H13,F15,A18,I4 WARM']
1/1 [==============================] - 0s 16ms/step
 
['F5,H8,H10,H13,F15,A18,I4 WARM ']
1/1 [==============================] - 0s 16ms/step
U
['F5,H8,H10,H13,F15,A18,I4 WARM U']
1/1 [==============================] - 0s 16ms/step
P
['F5,H8,H10,H13,F15,A18,I4 WARM UP']
1/1 [==============================] - 0s 16ms/step
 
['F5,H8,H10,H13,F15,A18,I4 WARM UP ']
1/1 [==============================] - 0s 15ms/step
O
['F5,H8,H10,H13,F15,A18,I4 WARM UP O']
1/1 [==============================] - 0s 16ms/step
F
['F5,H8,H10,H13,F15,A18,I4 WARM UP OF']
1/1 [==============================] - 0s 15ms/step
 
['F5,H8,H10,H13,F15,A18,I4 WARM UP OF ']
1/1 [=====

In [97]:


# why do we learn the same thing...is it diff at all?
def predict_char(text):
    texts = [text]
    #single 
    MAX_LENGTH = 128
    # for each text, continue predicting until we reach max length or end token
    final_texts = texts
    sequences = utils.texts_to_sequences(final_texts, char_to_int)
    data = pad_sequences(sequences, maxlen=SEQUENCE_LENGTH)
    pr = NAME_MODEL.predict(data)
    prob_thresh = np.max(pr,1) * 0.1
    pr[pr < prob_thresh]=0
    pr = (pr/pr.sum(axis=1,keepdims=1))[0]
    idxs = np.arange(len(pr))
    return np.random.choice(idxs, 1, p=pr)[0]
    
    #return predictions_list

In [98]:
p1 = predict_char("F5,H8,H10,H13,F15,A18,I4 ")
p2 = predict_char("A2,F8,F1,A24,I1 ")

1/1 [==============================] - 0s 15ms/step


In [99]:
p1

17

In [35]:
print(np.max(p1,1))
print(np.max(p2,1))

int_to_char[np.argmax(p1,1).item()]

[0.12556466]
[0.12516251]


'S'

In [49]:
p3=p1/p1.sum(axis=1,keepdims=1)
print(p3)

[[1.72423512e-07 1.76011341e-07 3.38816480e-03 3.71275768e-02
  1.89682178e-03 3.10630479e-04 3.80029231e-02 4.38466319e-04
  2.59014368e-02 4.95374389e-02 4.41518053e-02 3.66579667e-02
  4.90288399e-02 6.88316077e-02 1.53136428e-03 1.03672140e-03
  7.94554595e-04 6.83145896e-02 4.48931882e-04 2.91246525e-03
  1.98012199e-02 1.57413092e-02 8.30415338e-02 2.19754465e-02
  3.36246379e-02 2.25920841e-04 1.25564680e-01 2.81670373e-02
  4.55288013e-04 4.36146930e-02 6.04902506e-02 1.02318591e-02
  6.04689717e-02 1.16024725e-02 3.31582278e-02 1.41406571e-03
  1.15157766e-02 4.60461341e-03 1.56281923e-03 2.42658216e-03]]


In [55]:
p3[p3<0.01] = 0

In [57]:
p3 = p3/p3.sum(axis=1,keepdims=1)

In [58]:
p3

array([[0.        , 0.        , 0.        , 0.03801904, 0.        ,
        0.        , 0.0389154 , 0.        , 0.02652335, 0.05072687,
        0.04521192, 0.03753815, 0.05020606, 0.0704843 , 0.        ,
        0.        , 0.        , 0.06995487, 0.        , 0.        ,
        0.02027666, 0.01611927, 0.08503542, 0.02250309, 0.03443199,
        0.        , 0.12857959, 0.02884335, 0.        , 0.04466191,
        0.06194267, 0.01047753, 0.06192088, 0.01188106, 0.03395438,
        0.        , 0.01179228, 0.        , 0.        , 0.        ]],
      dtype=float32)